## Лекция 2  BM5    

### TfidfVectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
 
# инициализируем
vectorizer = TfidfVectorizer()

# составляем корпус документов
corpus = [
  'слово1 слово2 слово3',
  'слово2 слово3',
  'слово1 слово2 слово1',
  'слово4'
]

# считаем
X = vectorizer.fit_transform(corpus)
 
# получится следующая структура:
#        |  слово1  |  слово2  |  слово3  |  слово4
# текст1 |   0.6    |    0.5   |   0.6    |    0
# текст2 |   0      |    0.6   |   0.8    |    0
# текст3 |   0.9    |    0.4   |   0      |    0
# текст4 |   0      |    0     |   0      |    1
 
# чтобы получить сгенерированный словарь, из приведенной структуры TfidfVectorizer
# порядок совпадает с матрицей
vectorizer.get_feature_names()  # ['слово1', 'слово2', 'слово3', 'слово4']
 
# чтобы узнать индекс токена в словаре
vectorizer.vocabulary_.get('слово3') # вернет 2
 
# показать матрицу
X.toarray()
 
# теперь можно быстро подсчитать вектор для нового документа
new_doc = vectorizer.transform(['слово1 слово4 слово4']).toarray()  # результат [[0.36673901, 0, 0, 0.93032387]]
 

## Функция ранжирования bm25

Для обратного индекса есть общепринятая формула для ранжирования *Okapi best match 25* ([Okapi BM25](https://ru.wikipedia.org/wiki/Okapi_BM25)).    
Пусть дан запрос $Q$, содержащий слова  $q_1, ... , q_n$, тогда функция BM25 даёт следующую оценку релевантности документа $D$ запросу $Q$:

$$ score(D, Q) = \sum_{i}^{n} \text{IDF}(q_i)*\frac{TF(q_i,D)*(k+1)}{TF(q_i,D)+k(1-b+b\frac{l(d)}{avgdl})} $$ 
где   
>$TF(q_i,D)$ - частота слова $q_i$ в документе $D$      
$l(d)$ - длина документа (количество слов в нём)   
*avgdl* — средняя длина документа в коллекции    
$k$ и $b$ — свободные коэффициенты, обычно их выбирают как $k$=2.0 и $b$=0.75   
$$$$
$\text{IDF}(q_i)$ - это модернизированная версия IDF: 
$$\text{IDF}(q_i) = \log\frac{N-n(q_i)+0.5}{n(q_i)+0.5},$$
>> где $N$ - общее количество документов в коллекции   
$n(q_i)$ — количество документов, содержащих $q_i$

In [ ]:
Q = vectorizer.get_feature_names()

In [ ]:
### реализуйте эту функцию ранжирования 
from math import log

k = 2.0
b = 0.75


def bm25() -> float:
    return 

### __Задача 1__:    
Реализуйте поиск с метрикой *TF-IDF* через умножение матрицы на вектор.
Что должно быть в реализации:
- проиндексированная база, где каждый документ представлен в виде вектора TF-IDF
- функция перевода входяшего запроса в вектор по метрике TF-IDF
- ранжирование докуменов по близости к запросу по убыванию

В качестве корпуса возьмите корпус вопросов в РПН по Covid2019. Он состоит из:
> файл **answers_base.xlsx** - база ответов, у каждого ответа есть его номер, тематика и примеры вопросов, которые могут быть заданы к этому ответу. Сейчас проиндексировать надо именно примеры вопросов в качестве документов базы. Понимаете почему?

> файл **queries_base.xlsx** - вопросы юзеров, к каждому из которых проставлен номер верного ответа из базы. Разделите эти вопросы в пропорции 70/30 на обучающую (проиндексированную как база) и тестовую (как запросы) выборки. 


In [1]:
import pandas as pd

answers = pd.read_excel('answers_base.xlsx')
queries = pd.read_excel("queries_base.xlsx")

можно сделать препроцессинг, потому что в примере лежат слова по попрядку в документах без знаков препинания

In [2]:
import string
import re
import pandas as pd
from pymorphy2 import MorphAnalyzer


morph = MorphAnalyzer()

In [3]:
def preprocessing(data):
    data2 = []
    cnt = 0
    for v in data:
        strng = ''
        line = "".join(l for l in v if l not in (
            '.', ',', '?', '!', ':', ';', '—', '--', '<', '>', '"', '(', ')'))
        for word in line.split():
            word = re.sub(r'\ufeff', '', word)
            wrd = morph.parse(word)[0]
            strng += wrd.normal_form + ' '
        data2.append(strng)
    return data2

In [4]:
lemm_text = preprocessing(answers['Текст вопросов'])

In [5]:
answers['вопросы_препроцессинговые'] = lemm_text

In [6]:
answers.head()

,Номер связки,Текст вопросов,Текст ответа,Тематика,вопросы_препроцессинговые
0,57,У ребенка в школе продлили каникулы. Могу ли я...,Листок временной нетрудоспособности (больничны...,БОЛЬНИЧНЫЙ ЛИСТ,у ребёнок в школа продлить каникулы мочь ли я ...
1,78,Где сделать вакцинацию от коронавируса?\nСущес...,"Коронавирусы - это целое семейство вирусов, ко...",ВАКЦИНАЦИЯ,где сделать вакцинация от коронавирус существо...
2,326,Сколько стоит сделать вакцину от гриппа?\nМожн...,Бесплатно пройти вакцинацию можно в Вашей меди...,ВАКЦИНАЦИЯ,сколько стоить сделать вакцина от грипп можно ...
3,327,Могу я отказаться от вакцинации?\nВ каких случ...,Согласно приказу Министерства здравоохранения ...,ВАКЦИНАЦИЯ,мочь я отказаться от вакцинация в какой случай...
4,328,Безопасна ли вакцинация?\nОпасна ли вакцинация...,В соответствии с пунктами 1 и 2 статьи 12 Феде...,ВАКЦИНАЦИЯ,безопасный ли вакцинация опасный ли вакцинация...


In [7]:
queries.head()

,Текст вопроса,Номер связки,Тематика,Unnamed: 3,Unnamed: 4
0,с уважением Вероника Игоревна Ильич\n\nПосле ...,308.0,"ЗАКРЫТИЕ ГРАНИЦ, ОТКРЫТИЕ ГРАНИЦ РОССИИ И АВИА...",NaN,NaN
1,"Здравствуйте! Проинформируйте, пожалуйста, нуж...",324.0,ОРГАНИЗАЦИИ ОТДЫХА ДЕТЕЙ И ИХ ЗДОРОВЛЕНИЯ,NaN,NaN
2,"--\nДобрый день!\n Меня, Сидельникова Андрея...",57.0,БОЛЬНИЧНЫЙ ЛИСТ,NaN,NaN
3,Добрый день.\nВ Кемеровской области согласно п...,45.0,"ШТРАФЫ, НОРМАТИВНЫЕ АКТЫ И РЕКОМЕНДАЦИИ",NaN,NaN
4,"Здравствуйте, в моем городе Кострома введено о...",3.0,"ШТРАФЫ, НОРМАТИВНЫЕ АКТЫ И РЕКОМЕНДАЦИИ",NaN,NaN


In [8]:
from sklearn.model_selection import train_test_split

RANDOM_STATE = 350

train, test = train_test_split(queries, test_size=0.3,
                               random_state=RANDOM_STATE)

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
 
# инициализируем
vectorizer = TfidfVectorizer()

In [22]:
X = vectorizer.fit_transform(answers['вопросы_препроцессинговые'])

In [23]:
X

<43x1020 sparse matrix of type '<class 'numpy.float64'>'
	with 2211 stored elements in Compressed Sparse Row format>

In [24]:
matr_answers = X.toarray()

In [25]:
matr_answers

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.25150849]])

In [26]:
import numpy as np

In [38]:
def text_to_vec(text, vectorizer, matr):
    new_doc = vectorizer.transform(text).toarray()
    new_matr = matr.dot(new_doc[0])
    return new_matr

In [39]:
text = text_to_vec(queries.head(1)['Текст вопроса'], vectorizer, matr_answers)

In [40]:
text

array([0.06272099, 0.02216119, 0.        , 0.04690233, 0.        ,
       0.        , 0.01970294, 0.03542588, 0.        , 0.07079659,
       0.08353944, 0.15906375, 0.03332326, 0.        , 0.1731618 ,
       0.        , 0.06055218, 0.        , 0.        , 0.07709556,
       0.04607895, 0.06248688, 0.18022558, 0.05330041, 0.        ,
       0.22392487, 0.05469213, 0.08125526, 0.08768456, 0.        ,
       0.01788488, 0.1642192 , 0.08835104, 0.        , 0.0173241 ,
       0.1294703 , 0.02264223, 0.16993824, 0.20911701, 0.12971927,
       0.07896762, 0.02926936, 0.06568891])

In [41]:
n_conn = answers['Номер связки'].tolist()

In [42]:
def ranging(text):
    dict_texts = {}

    cnt = 0
    for x in np.nditer(text):
        dict_texts[float(x)] = n_conn[cnt]
        cnt += 1

    new_dict = sorted(dict_texts.keys(), reverse=True)

    first = []
    print('Ранжированные документы по близости: \n---\n')
    for k in new_dict:
        first.append(dict_texts[k])
        print (k, ': номер связки - ', dict_texts[k])

    strng = '\n---\nДля этого текста связка самого близкого ответа: ' + str(first[0])
    print(strng)

In [43]:
ranging(text)

Ранжированные документы по близости: 
---

0.2239248671194552 : номер связки -  37
0.2091170077298064 : номер связки -  135
0.18022557722602078 : номер связки -  286
0.17316179640354723 : номер связки -  1
0.16993824318245715 : номер связки -  6
0.16421919848158378 : номер связки -  270
0.1590637515984202 : номер связки -  308
0.12971926975037168 : номер связки -  5
0.12947029742191746 : номер связки -  88
0.08835104394916726 : номер связки -  246
0.08768456332602703 : номер связки -  132
0.08353944250300764 : номер связки -  173
0.08125526125642152 : номер связки -  74
0.07896761992493447 : номер связки -  3
0.07709556090241905 : номер связки -  34
0.07079658505465908 : номер связки -  89
0.0656889099571096 : номер связки -  21
0.06272098684490689 : номер связки -  57
0.06248688376800135 : номер связки -  325
0.060552175597895684 : номер связки -  10
0.054692134687624856 : номер связки -  32
0.053300409558021955 : номер связки -  79
0.046902326648256054 : номер связки -  327
0.0460789

Но данные без препроцессинга иногда могут быть ближе к реальности

In [33]:
vectorizer_2 = TfidfVectorizer()

In [34]:
X_start = vectorizer_2.fit_transform(answers['Текст вопросов'])

In [35]:
X_start

<43x1554 sparse matrix of type '<class 'numpy.float64'>'
	with 2647 stored elements in Compressed Sparse Row format>

In [44]:
matr_answers_2 = X_start.toarray()

In [45]:
matr_answers_2

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.23484538]])

In [48]:
text_2 = text_to_vec(queries.head(1)['Текст вопроса'], vectorizer_2, matr_answers_2)

In [49]:
text_2

array([0.05132964, 0.00826135, 0.        , 0.02444088, 0.        ,
       0.        , 0.01655842, 0.02387181, 0.        , 0.05062162,
       0.04710088, 0.18612481, 0.02986892, 0.        , 0.12088349,
       0.        , 0.04099694, 0.        , 0.        , 0.0479808 ,
       0.02159938, 0.06524996, 0.12684607, 0.05579022, 0.        ,
       0.1043222 , 0.0249666 , 0.06173973, 0.05591414, 0.        ,
       0.01440677, 0.12635103, 0.07085367, 0.        , 0.01180225,
       0.11765951, 0.02340343, 0.14332734, 0.16869162, 0.11482521,
       0.02310699, 0.02007746, 0.01955076])

In [50]:
ranging(text_2)

Ранжированные документы по близости: 
---

0.18612481376297696 : номер связки -  308
0.16869161558362822 : номер связки -  135
0.14332733589020907 : номер связки -  6
0.12684606813855775 : номер связки -  286
0.12635102520844527 : номер связки -  270
0.12088349071349433 : номер связки -  1
0.11765950736340813 : номер связки -  88
0.11482520646588391 : номер связки -  5
0.10432220238297966 : номер связки -  37
0.07085366550799661 : номер связки -  246
0.0652499567437455 : номер связки -  325
0.061739732054359675 : номер связки -  74
0.05591413582628506 : номер связки -  132
0.055790216712495955 : номер связки -  79
0.05132963940168474 : номер связки -  57
0.050621616816445836 : номер связки -  89
0.0479807957433737 : номер связки -  34
0.047100880565229285 : номер связки -  173
0.04099693804065354 : номер связки -  10
0.029868921992849802 : номер связки -  46
0.024966597718604842 : номер связки -  32
0.024440881584028892 : номер связки -  327
0.02387181219277522 : номер связки -  43
0.0

### __Задача 2__:    
Аналогичная задаче1 с другой метрикой 

Реализуйте поиск с метрикой *BM25* через умножение матрицы на вектор. Что должно быть в реализации:

- проиндексированная база, где каждый документ представлен в виде вектора BM25
- функция перевода входяшего запроса в вектор по метрике BM25
- ранжирование докуменов по близости к запросу по убыванию

In [ ]:
from gensim.summarization.bm25 import get_bm25_weights

In [ ]:
result = get_bm25_weights(answers['Текст вопросов'], n_jobs=-1)

In [ ]:
result